In [149]:
import os
import json
import xml.dom.minidom

from tqdm import tqdm
from glob import glob
from xml.dom.minidom import parse

In [150]:
def xml_to_json(xml_path, json_path):
    
    """
    imageHeight:
    imageWidth:
    points:所有xml中的slab坐标
    imagePath:文件名称
    
    """
    for name in tqdm(glob(os.path.join(xml_path, '*.xml'))):
        points = []
        DOMTree = xml.dom.minidom.parse(name)
        collection = DOMTree.documentElement
        imagePath = collection.getElementsByTagName("filename")[0].childNodes[0].data
        
        sizes = collection.getElementsByTagName("size")
        for size in sizes:
            imageHeight = int(size.getElementsByTagName("height")[0].childNodes[0].data)
            imageWidth = int(size.getElementsByTagName("width")[0].childNodes[0].data)
        
        objs = collection.getElementsByTagName("object")
        for obj in objs:
            name = obj.getElementsByTagName("name")[0].childNodes[0].data
            bndboxs = obj.getElementsByTagName("bndbox")
            for bndboxs in bndboxs:
                xmin = int(bndboxs.getElementsByTagName("xmin")[0].childNodes[0].data)
                ymin = int(bndboxs.getElementsByTagName("ymin")[0].childNodes[0].data)
                xmax = int(bndboxs.getElementsByTagName("xmax")[0].childNodes[0].data) 
                ymax = int(bndboxs.getElementsByTagName("ymax")[0].childNodes[0].data)
            points.append((name,[[xmin, ymin],[xmax, ymin],[xmax, ymax],[xmin, ymax]]))    

        new_json(json_path, imagePath, imageHeight, imageWidth)
        
        add_info(json_path, imagePath, points)
        


    
       

In [166]:
def new_json(path, imagePath, imageHeight, imageWidth):
    json_info = {"version": "3.16.7", 
                     "flags": {}, 
                     "shapes": [],
                     "lineColor": [0, 255, 0, 128],
                     "fillColor": [255, 0, 0, 128], 
                     "imagePath": "{}".format(imagePath),
                     "imageHeight": imageHeight, 
                     "imageWidth": imageWidth
                    }   
    with open(os.path.join(path, '{}.json'.format(imagePath[:-4])), 'w') as f: 
        json.dump(json_info,f)

In [185]:
def add_info(path, imagePath, points):
    name = os.path.join(path, '{}.json'.format(imagePath[:-4]))
    with open(name, 'r') as f:
        ann = json.load(f)
    for point in points:
        json_info = {"label": "{}".format(point[0]),
                          "line_color": None,
                          "fill_color": None,
                          "points": point[1] ,
                          "shape_type": "polygon",
                          "flags": {}
                        }   
        ann["shapes"].append(json_info)
    
    with open(name, 'w') as f:
        json.dump(ann, f)

In [186]:
xml_to_json("D:/data/total_background/total_xml/", "D:/data/total_background/total_json/")

100%|███████████████████████████████████████████████████████████████████████████████| 778/778 [00:00<00:00, 804.07it/s]
